In [1]:
from tensorflow.python.keras.models import load_model
import pandas as pd
import cv2
import os
import numpy as np
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
load_model = load_model("model/cnn_best_model_20240405_tuning_1_창측제외_테스트변경.h5")

C:\Users\witlab\anaconda3\envs\gpu_test_python3.6\lib\site-packages\tensorflow\python\framework\dtypes.py:469: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\witlab\anaconda3\envs\gpu_test_python3.6\lib\site-packages\tensorflow\python\framework\dtypes.py:470: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\witlab\anaconda3\envs\gpu_test_python3.6\lib\site-packages\tensorflow\python\framework\dtypes.py:471: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\witlab\anaconda3\envs\

In [2]:
test_image_path = r'image\auto_intg\preprocess_no_cut\20240407'
test_image_files = [f for f in os.listdir(test_image_path) if f.endswith('.jpg')]

test_images = []
for file in test_image_files:
    img = cv2.imread(os.path.join(test_image_path, file))
    if img is None:
        print(f"{file} 읽을 수 없음")
        continue

    img = img / 255.0  # 0~1로 정규화
    test_images.append(img)

test_images = np.array(test_images)
print(test_images.shape)

(735, 224, 224, 3)


In [3]:
# test 조도 데이터 불러오기

test_excel_path = 'data/20240407_img_illum.xlsx'
test_df = pd.read_excel(test_excel_path, engine='openpyxl')

# illum_columns = [f'illum_{i}' for i in range(1, 10)]
illum_columns = [f'new_illum_{i}' for i in [10, 11, 12, 4, 5, 6, 1, 2, 3, 7, 8, 9]]

# Extract output (illum) columns
test_target_data = (test_df[illum_columns]).to_numpy()
test_target_data[test_target_data < 0] = 0
print(test_target_data.shape)

pred = load_model.predict(test_images)

(735, 12)


In [4]:
# Create DataFrames from the arrays
actual_df = pd.DataFrame(test_target_data, columns=[f'Actual_Illum_{i}' for i in [10, 11, 12, 4, 5, 6, 1, 2, 3, 7, 8, 9]])
predicted_df = pd.DataFrame(pred, columns=[f'Predicted_Illum_{i}' for i in [10, 11, 12, 4, 5, 6, 1, 2, 3, 7, 8, 9]])

# Concatenate the DataFrames horizontally
comparison_data = pd.concat([actual_df, predicted_df], axis=1)

# Save the DataFrame to an Excel file
excel_output_path = 'cnn_results_20240407_test_20240408_테스트변경모델.xlsx'
comparison_data.to_excel(excel_output_path, index=False)

print(f"Comparison results saved to {excel_output_path}")

Comparison results saved to cnn_results_20240407_test_20240408_테스트변경모델.xlsx


In [5]:
from sklearn.metrics import r2_score, mean_absolute_error
r_squared = r2_score(test_target_data, pred)
mae = mean_absolute_error(test_target_data, pred)

print("Mean Absolute Error (MAE):", mae)
print("R-squared (R²):", r_squared)

Mean Absolute Error (MAE): 23.715568676696424
R-squared (R²): 0.9800896215389742
